# LU decomposition
My notes for UCL [Math0058 - Computational Methods](https://tbetcke.github.io/math0058_lecture_notes/python_lu_decomposition.html)

### Setup

In [72]:
import numpy as np
import scipy as sp

np.random.seed(0)
np.set_printoptions(precision=2)

In [73]:
def generate_matrix(n):
    M = np.random.randint(low=1, high=10, size=(n,n))
    return M.astype(np.float64)

A = generate_matrix(4)
print(A)

[[6. 1. 4. 4.]
 [8. 4. 6. 3.]
 [5. 8. 7. 9.]
 [9. 2. 7. 8.]]


### The LU algorithm

In [74]:
def lu(A):
    """Return the LU decomposition of A, without modifying it."""
    if not (n := A.shape[0]) == A.shape[1]:
        raise ValueError("Input matrix must be square.")

    U = np.copy(A)
    L = np.identity(n, dtype=np.float64)

    for col in range(n-1):
        for row in range(col + 1, n):
            L[row, col] = U[row, col] / U[col, col]
            U[row, col:] -= L[row, col] * U[col, col:]
            U[row, col] = 0

    return (L, U)

In [75]:
L, U = lu(A)
assert np.allclose(L @ U, A)
print(L)
print(U)

[[1.   0.   0.   0.  ]
 [1.33 1.   0.   0.  ]
 [0.83 2.69 1.   0.  ]
 [1.5  0.19 0.47 1.  ]]
[[ 6.    1.    4.    4.  ]
 [ 0.    2.67  0.67 -2.33]
 [ 0.    0.    1.87 11.94]
 [ 0.    0.    0.   -3.13]]


### Speed comparison
Obviously we should avoid doing calculation in python at all costs.

In [76]:
def compare_lu(n):
    A = generate_matrix(n)
    python_lu = %timeit -q -o lu(A)
    scipy_lu = %timeit -q -o sp.linalg.lu(A)
    print(f"Python time: {python_lu}")
    print(f"Scipy time: {scipy_lu}")
    print(f"Ratio: {python_lu.best/scipy_lu.best}")

For small `n` our implementation is fine.

In [77]:
compare_lu(5)

Python time: 36.3 µs ± 925 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
Scipy time: 17.7 µs ± 641 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
Ratio: 2.034737396941132


But for large `n` it is very slow.

In [78]:
compare_lu(500)

Python time: 398 ms ± 8.67 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Scipy time: 9.28 ms ± 1.05 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
Ratio: 47.221089904729716


### Determinants
Once we have found the LU decomposition, it can be used to cheaply find the determinant.

In [79]:
detA = sp.linalg.det(A)
detL = L.diagonal().prod()
detU = U.diagonal().prod()
assert np.isclose(detA, detL * detU)
print(detA)


-94.00000000000003
